In [4]:
import pandas as pd
import numpy as np

import lightgbm as lgb
import xgboost as xgb

from sklearn.model_selection import train_test_split  # type: ignore
from sklearn.linear_model import LinearRegression  # type: ignore
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import time
import matplotlib.pyplot as plt

import json

from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import dask.dataframe as dd
import os

dataset_params = {
        "SIFT100M": {
            "M": 32,
            "efC": 500,
            "efS": 500,
            "li": 1,
            "label": "SIFT100M",
        },
        "GIST1M": {
            "M": 32,
            "efC": 500,
            "efS": 1000,
            "li": 1,
            "label": "GIST1M",
        },
        "GLOVE100": {
            "M": 16,
            "efC": 500,
            "efS": 500,
            "li": 1,
            "label": "GLOVE1M",
        },
        "DEEP100M":{
            "M": 32,
            "efC": 500,
            "efS": 750,
            "li": 1,
            "label": "DEEP100M",
        },
        "T2I100M":{
            "M": 80,
            "efC": 1000,
            "efS": 2500,
            "li": 2,
            "label": "T2I100M",
        }
    }

SEED = 42

index_metric_feats = ["step", "dists", "inserts"]
neighbor_distances_feats = ["first_nn_dist", "nn_dist", "furthest_dist"]
neighbor_stats_feats = ["avg_dist", "variance", "percentile_25", "percentile_50", "percentile_75"]
data_dims_feats = ["dim_l2_norm", "dim_l1_norm", "dim_mean", "dim_median", "dim_std", "dim_var", "dim_min", "dim_max", "dim_range", "dim_energy", "dim_skewness", "dim_kurtosis", "dim_perc_25", "dim_perc_75", "dim_perc_95"]
neighbor_stats_feats_new = ["std", "range", "energy", "skewness", "kurtosis", "percentile_95"]

In [5]:
ds_name = "SIFT100M"
k = 50
li = 5
queries = 10000
M = dataset_params[ds_name]["M"]
efC = dataset_params[ds_name]["efC"]
efS = dataset_params[ds_name]["efS"]

columns_to_load = ["qid", "elaps_ms"] + index_metric_feats + neighbor_distances_feats + neighbor_stats_feats + data_dims_feats + neighbor_stats_feats_new + ["r", "feats_collect_time_ms"]

datapath = f"/data/mchatzakis/et_training_data/early-stop-training/{ds_name}/k{k}/M{M}_efC{efC}_efS{efS}_qs{queries}_li{li}_imp.txt"
all_queries_dask = dd.read_csv(datapath, usecols=columns_to_load)
all_queries_data = all_queries_dask.compute()

In [6]:
all_queries_data.head()

,qid,step,dists,elaps_ms,inserts,first_nn_dist,nn_dist,avg_dist,furthest_dist,percentile_25,...,dim_max,dim_range,dim_energy,dim_skewness,dim_kurtosis,dim_perc_25,dim_perc_75,dim_perc_95,feats_collect_time_ms,r
0,0,0,50,0.451088,51,105106.0,102107.0,135078.984375,165557.0,124807.0,...,157.0,157.0,258253.0,2.075178,3.977462,0.0,34.0,101.0,0.377893,0.0
1,0,0,55,0.872135,56,105106.0,97570.0,132397.875000,153798.0,123862.0,...,157.0,157.0,258253.0,2.075178,3.977462,0.0,34.0,101.0,0.099897,0.0
2,0,1,60,1.021147,60,105106.0,97570.0,129950.156250,149828.0,123205.0,...,157.0,157.0,258253.0,2.075178,3.977462,0.0,34.0,101.0,0.094891,0.0
3,0,1,65,1.155138,62,105106.0,97570.0,129732.937500,149222.0,123205.0,...,157.0,157.0,258253.0,2.075178,3.977462,0.0,34.0,101.0,0.095844,0.0
4,0,1,70,1.287937,65,105106.0,97534.0,127782.460938,147794.0,118932.0,...,157.0,157.0,258253.0,2.075178,3.977462,0.0,34.0,101.0,0.094891,0.0


In [8]:
all_queries_data.sample(10)

,qid,step,dists,elaps_ms,inserts,first_nn_dist,nn_dist,avg_dist,furthest_dist,percentile_25,...,dim_max,dim_range,dim_energy,dim_skewness,dim_kurtosis,dim_perc_25,dim_perc_75,dim_perc_95,feats_collect_time_ms,r
98783,8307,9,565,17.302036,206,82328.0,47425.0,66128.078125,72886.0,63738.0,...,117.0,117.0,257445.0,1.203448,0.478845,5.0,44.0,102.0,0.101089,0.42
124437,144,43,1640,46.103001,254,42975.0,28616.0,33626.859375,36121.0,32784.0,...,137.0,137.0,258871.0,1.858197,2.337068,0.0,26.0,136.0,0.093937,0.86
44402,9480,485,8180,217.138052,177,5248.0,3970.0,4956.240234,5415.0,4702.0,...,206.0,206.0,260949.0,3.374529,11.116528,0.0,7.0,111.0,0.088930,1.00
120431,253,234,11615,318.962097,309,99760.0,60975.0,70772.320312,74953.0,69202.0,...,121.0,121.0,257721.0,1.436238,1.451099,6.0,47.0,121.0,0.095844,0.98
12314,1096,93,3810,103.178978,242,40509.0,30693.0,41181.019531,44167.0,39512.0,...,125.0,125.0,258295.0,1.604243,1.554233,2.0,37.0,125.0,0.091076,0.98
117015,630,187,9610,258.877039,268,85451.0,55664.0,71444.257812,76161.0,69020.0,...,128.0,128.0,258432.0,1.477446,1.293972,4.0,45.0,113.0,0.097036,0.86
71426,8893,451,14325,321.671963,207,43948.0,31908.0,43908.558594,49109.0,42698.0,...,136.0,136.0,258572.0,1.812500,2.287206,0.0,34.0,136.0,0.056028,1.00
69875,8180,300,8720,234.745026,273,40096.0,22423.0,28330.779297,30477.0,27489.0,...,182.0,182.0,258762.0,2.578554,6.937395,0.0,29.0,95.0,0.090837,1.00
127178,3376,216,5210,141.470194,185,20926.0,13026.0,16736.679688,18228.0,16136.0,...,148.0,148.0,259921.0,2.243611,4.005967,1.0,27.0,148.0,0.092030,1.00
99311,5782,29,1245,36.614895,200,47796.0,31868.0,37836.761719,41553.0,36325.0,...,134.0,134.0,259419.0,1.776762,2.221609,3.0,32.0,134.0,0.097990,0.76


In [7]:
all_queries_data.describe()

,qid,step,dists,elaps_ms,inserts,first_nn_dist,nn_dist,avg_dist,furthest_dist,percentile_25,...,dim_max,dim_range,dim_energy,dim_skewness,dim_kurtosis,dim_perc_25,dim_perc_75,dim_perc_95,feats_collect_time_ms,r
count,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,...,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07,3.199616e+07
mean,4.964227e+03,2.345386e+02,8.810880e+03,2.395019e+02,2.352847e+02,6.365877e+04,4.068568e+04,5.277159e+04,5.695605e+04,5.091449e+04,...,1.301244e+02,1.301213e+02,2.586582e+05,1.665670e+00,1.973213e+00,2.678172e+00,3.728851e+01,1.209748e+02,9.324840e-02,9.206981e-01
std,2.974072e+03,1.460507e+02,5.693646e+03,1.531011e+02,4.625110e+01,2.434873e+04,1.531718e+04,1.772782e+04,1.878797e+04,1.730984e+04,...,1.499027e+01,1.499319e+01,6.637837e+02,3.784789e-01,1.570998e+00,2.658499e+00,8.612538e+00,1.181026e+01,1.021507e-01,1.705124e-01
min,0.000000e+00,0.000000e+00,5.000000e+01,2.479550e-01,5.000000e+01,1.213000e+03,7.750000e+02,1.443900e+03,1.625000e+03,1.323000e+03,...,1.050000e+02,1.050000e+02,2.566820e+05,8.195300e-01,-4.845140e-01,0.000000e+00,3.000000e+00,7.600000e+01,5.078300e-02,0.000000e+00
25%,2.276000e+03,1.050000e+02,4.045000e+03,1.121721e+02,2.050000e+02,4.752600e+04,3.028700e+04,4.174332e+04,4.550700e+04,4.003200e+04,...,1.200000e+02,1.200000e+02,2.581770e+05,1.407713e+00,9.718510e-01,1.000000e+00,3.200000e+01,1.130000e+02,9.203000e-02,9.400000e-01
50%,5.101000e+03,2.270000e+02,8.155000e+03,2.226391e+02,2.350000e+02,6.489800e+04,4.163800e+04,5.484562e+04,5.919500e+04,5.287200e+04,...,1.270000e+02,1.270000e+02,2.586190e+05,1.602661e+00,1.573328e+00,2.000000e+00,3.800000e+01,1.200000e+02,9.393700e-02,1.000000e+00
75%,7.603000e+03,3.600000e+02,1.294000e+04,3.492939e+02,2.660000e+02,7.982500e+04,5.169800e+04,6.541544e+04,7.014600e+04,6.333600e+04,...,1.370000e+02,1.370000e+02,2.591100e+05,1.849003e+00,2.498860e+00,4.000000e+00,4.300000e+01,1.280000e+02,9.608300e-02,1.000000e+00
max,9.999000e+03,5.230000e+02,2.700500e+04,9.713650e+02,4.860000e+02,1.657380e+05,1.620610e+05,2.085762e+05,2.719220e+05,1.888340e+05,...,2.220000e+02,2.220000e+02,2.615150e+05,3.874847e+00,1.472042e+01,2.100000e+01,5.800000e+01,1.770000e+02,5.046582e+01,1.000000e+00


In [11]:
# Average collection time:
all_queries_data["feats_collect_time_ms"].mean()

0.09324840195345321